In [1]:
import importlib
importlib.invalidate_caches()

In [2]:
import os
os.environ["OPENAI_API_BASE"]= "https://api.openai.com/v1"

In [3]:
import os
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google import genai
from google.adk.models.lite_llm import LiteLlm
import litellm
import os

AGENT_MODEL = LiteLlm(model="openai/gpt-4o-mini")

In [4]:
# Install and import required libraries
import nest_asyncio
import asyncio
nest_asyncio.apply()  # Required for async in notebooks

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Constants — define application, user, and session identifiers
APP_NAME      = "adk_wew_app"
USER_ID       = "wew"
SESSION_ID    = "support_session"

In [5]:
# FAQ knowledge base & tool 
FAQ_DATA = {
    "return policy": "You can return items within 30 days of purchase.",
    "hours": "Our support team is available from 9 am to 5 pm, Monday to Friday.",
    "contact": "You can reach support at help@example.com."
}

def lookup_faq(question: str) -> str:
    faq_text = "\n".join(f"- {k}: {v}" for k, v in FAQ_DATA.items())
    prompt = (
        f"You are a helpful assistant. Here is a list of FAQs:\n\n{faq_text}\n\n"
        f"User question: \"{question}\". "
        f"Reply with the best match or say you don't know."
    )
    response = litellm.completion(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"].strip()

faq_tool  = FunctionTool(func=lookup_faq)

In [6]:
# Specialist Agents
greeting_agent = LlmAgent(
    name="GreetingAgent",
    description="Handles greetings from users.",
    instruction="Respond cheerfully when the user says hello.",
    model=AGENT_MODEL
)

account_agent = LlmAgent(
    name="AccountAgent",
    description="Handles questions about login issues or account access.",
    instruction="Help users who are having trouble logging in or accessing their account.",
    model=AGENT_MODEL
)

faq_agent = LlmAgent(
    name="FAQAgent",
    description="Answers common questions using the FAQ knowledge base.",
    instruction="Use the FAQ tool to answer questions that match the FAQs.",
    model=AGENT_MODEL,
    tools=[faq_tool]
)

# Root agent with delegation logic
root_agent = LlmAgent(
    name="SupportRootAgent",
    description="Delegates to specialized sub-agents for support queries.",
    instruction=(
        "If the user greets you, delegate to GreetingAgent.\n"
        "If the user has an account or login issue, delegate to AccountAgent.\n"
        "If the question matches a known FAQ topic (e.g., returns, hours, contact), "
        "delegate to FAQAgent. Do not answer as the FAQAgent if the topic doesn't match any of the FAQs.\n"
        "Otherwise, answer directly as best you (the Root Agent) can."
    ),
    model=AGENT_MODEL,
    sub_agents=[greeting_agent, account_agent, faq_agent]
)

In [7]:
# Session & runner 
session_service = InMemorySessionService()
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID,
                               session_id=SESSION_ID)

runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

# Function to chat with the root agent
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")
    content = types.Content(role="user", parts=[types.Part(text=query)])
    final_response = "Agent did not produce a final response."

    async for event in runner.run_async(user_id=USER_ID,
                                        session_id=SESSION_ID,
                                        new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response = event.content.parts[0].text
            break

    print(f"<<< Agent {event.author}'s response: {final_response}")

# Test the full system
await call_agent_async("Hello!")                       # GreetingAgent
await call_agent_async("I can't access my account.")   # AccountAgent
await call_agent_async("What is your return policy?")  # FAQAgent
await call_agent_async("I have a privacy question.")   # SupportRootAgent


>>> User Query: Hello!
<<< Agent GreetingAgent's response: Hello there! 😊 How can I brighten your day today?

>>> User Query: I can't access my account.
<<< Agent AccountAgent's response: I'm here to help you with your account access issue. Could you please provide more details about the problem you're facing? For instance, are you receiving any error messages, or have you forgotten your password?

>>> User Query: What is your return policy?
<<< Agent FAQAgent's response: Our return policy allows you to return items within 30 days of purchase.

>>> User Query: I have a privacy question.
<<< Agent SupportRootAgent's response: I'm here to help! Could you please provide more details about your privacy question?
